In [1]:
import pandas as pd

# Load Reviews
reviews_df = pd.read_json(r"C:\Users\HP\Documents\BSc in Data Science\Year 2\Semester 4\Data Mining & Machine Learning\All_Beauty.json\All_Beauty.json", lines=True)

# Load Metadata
metadata_df = pd.read_json(r"C:\Users\HP\Documents\BSc in Data Science\Year 2\Semester 4\Data Mining & Machine Learning\meta_All_Beauty.json\meta_All_Beauty.json", lines=True)


In [2]:
# Convert reviewTime to datetime (assuming reviewTime is a string like "MM DD, YYYY")
reviews_df['reviewTime'] = pd.to_datetime(reviews_df['reviewTime'], format='%m %d, %Y')

# If instead using unixReviewTime:
# reviews_df['reviewTime'] = pd.to_datetime(reviews_df['unixReviewTime'], unit='s')

# Filter reviews to only those in 2017 or 2018
mask = (reviews_df['reviewTime'] >= '2017-01-01') & (reviews_df['reviewTime'] <= '2018-12-31')
reviews_17_18 = reviews_df.loc[mask].copy()


In [3]:
print(reviews_17_18['reviewTime'].min(), "to", reviews_17_18['reviewTime'].max())
print("Number of reviews (2017-2018):", len(reviews_17_18))


2017-01-01 00:00:00 to 2018-10-02 00:00:00
Number of reviews (2017-2018): 100631


In [4]:
# Merge the reviews with metadata on 'asin'
combined_df = pd.merge(reviews_17_18, metadata_df, on='asin', how='inner')

# Check the result
print("Combined shape:", combined_df.shape)
print(combined_df.columns)


Combined shape: (103414, 30)
Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image', 'category', 'tech1', 'description', 'fit', 'title',
       'also_buy', 'tech2', 'brand', 'feature', 'rank', 'also_view', 'details',
       'main_cat', 'similar_item', 'date', 'price', 'imageURL',
       'imageURLHighRes'],
      dtype='object')


In [5]:
# Select only the needed columns
combined_df = combined_df[['asin', 'reviewText', 'summary', 'price', 'brand', 'title']]

# Optional: drop any remaining missing values (if any)
combined_df = combined_df.dropna(subset=['price','brand','title'])


In [7]:
combined_df.head()

,asin,reviewText,summary,price,brand,title
0,9790787006,I love Jenna ! But this just out right sucks !...,Old baby powder,$13.85,Jenna Jameson,Jenna Jameson Heartbreaker Perfume for women 3...
1,9790787006,I really want to thank the writers of fragranc...,Sexy and Comforting...All In One!,$13.85,Jenna Jameson,Jenna Jameson Heartbreaker Perfume for women 3...
2,9790787006,I'm sorry but this perfume is awful! It's too ...,Dont waste your money,$13.85,Jenna Jameson,Jenna Jameson Heartbreaker Perfume for women 3...
3,9790787006,I love it when my wife wears this perfume. It'...,I love it when my wife wears this perfume,$13.85,Jenna Jameson,Jenna Jameson Heartbreaker Perfume for women 3...
4,9790787006,"At first spray, it's kinda, ummhhmm? Then as i...","Love it, keep selling!",$13.85,Jenna Jameson,Jenna Jameson Heartbreaker Perfume for women 3...


In [8]:
combined_df.shape

(103414, 6)

In [9]:
combined_df.isnull().sum()


asin            0
reviewText    153
summary        89
price           0
brand           0
title           0
dtype: int64

In [16]:
# 1. Count how many times each title appears
title_counts = combined_df['title'].value_counts()

# 2. Get titles that appear more than 20 times
titles_to_keep = title_counts[title_counts > 20].index

# 3. Filter the DataFrame to keep only those rows
filtered_df = combined_df[combined_df['title'].isin(titles_to_keep)]

filtered_df.shape


(63079, 6)

In [17]:
# Export the combined DataFrame to CSV
filtered_df.to_csv('amazon_reviews_2017_2018_with_metadata.csv', index=False)
